In [ ]:
import nglview
import openmm.app
from openff.toolkit import ForceField, Molecule
from openff.units import ensure_quantity, unit

from openff.interchange import Interchange
from openff.interchange.interop.internal.gromacs import _get_virtual_site_positions
from openff.interchange.interop.openmm._positions import to_openmm_positions

In [ ]:
def viz(smiles: str) -> nglview.NGLWidget:
    molecule = Molecule.from_smiles(smiles)
    molecule.generate_conformers(n_conformers=1)

    # TODO: Replace this experimental force field with examples of virtual sites created via the Toolkit API
    interchange = Interchange.from_smirnoff(
        ForceField(
            "/Users/mattthompson/Downloads/force-field.offxml",
            allow_cosmetic_attributes=True,
        ),
        [molecule],
    )

    positions: unit.Quantity = to_openmm_positions(interchange)

    n_atoms = interchange.topology.n_atoms

    for index, virtual_site_key in enumerate(interchange["VirtualSites"].slot_map):
        # TODO: Merge this functionality from GROMACS to a common module
        positions[n_atoms + index] = _get_virtual_site_positions(
            virtual_site_key, interchange
        )

    with open("_tmp.pdb", "w") as file:
        openmm.app.PDBFile.writeFile(
            topology=interchange.to_openmm_topology(),
            positions=ensure_quantity(positions, "openmm"),
            file=file,
        )

    view = nglview.show_file("_tmp.pdb")

    view.clear_representations()
    view.add_representation("ball+stick", aspectRatio=3)

    return view

In [ ]:
viz("c1ncccc1")

In [ ]:
viz("BrCCCC")